In [ ]:
# default_exp models.game_data

In [ ]:
#exporti

import statsapi as mlb
from pydantic import (
    BaseModel,
    Field,
    validator,
    conint,
    constr,
    root_validator
)
from MLB_DataDevTools.models.base_models import *
from enum import Enum
import datetime as dt
from typing import Optional,Dict

In [ ]:
from MLB_DataDevTools.mlb_api import get_season_game_pks
from MLB_DataDevTools.database import create_mlb_engine
import pandas as pd

engine = create_mlb_engine()

games = pd.read_sql_table('game',engine.connect())

games.columns

Index(['game_pk', 'game_type', 'game_doubleHeader', 'game_id',
       'game_gamedayType', 'game_tiebreaker', 'game_gameNumber',
       'game_calendarEventID', 'game_season', 'game_seasonDisplay',
       'datetime_dateTime', 'datetime_originalDate', 'datetime_dayNight',
       'datetime_time', 'datetime_ampm', 'status_abstractGameState',
       'status_codedGameState', 'status_detailedState', 'status_statusCode',
       'status_abstractGameCode', 'teams_away_id', 'teams_home_id', 'venue_id',
       'weather_condition', 'weather_temp', 'weather_wind',
       'review_hasChallenges', 'review_away_used', 'review_away_remaining',
       'review_home_used', 'review_home_remaining', 'flags_noHitter',
       'flags_perfectGame', 'flags_awayTeamNoHitter',
       'flags_awayTeamPerfectGame', 'flags_homeTeamNoHitter',
       'flags_homeTeamPerfectGame', 'probablePitchers_away_id',
       'probablePitchers_home_id', 'officialScorer_id', 'primaryDatacaster_id',
       'secondaryDatacaster_id', 'stat

In [ ]:
gamePks = get_season_game_pks(2019)

In [ ]:
example_game = mlb.get('game',{'gamePk':gamePks[-100]})
example_game['gamePk']

566547

In [ ]:
example_game.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [ ]:
gameData = example_game['gameData']

## Game Details

In [ ]:
gameData['game']

{'pk': 566547,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/09/25/colmlb-sfnmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566547-2019-09-25',
 'season': '2019',
 'seasonDisplay': '2019'}

In [ ]:
#exporti 

class GameDetails(BaseModel):
    pk: int
    type: GameType
    doubleHeader: bool
    id: str
    gamedayType: GamedayType
    tiebreaker: bool
    gameNumber: conint(gt=0,lt=3)
    calendarEventID: str
    season: str
    seasonDisplay: str
    
    @validator('doubleHeader')
    def validate_double_header(cls,v):
        if v == 'S':
            return True
        else:
            return v

In [ ]:
gameData['game']

{'pk': 566547,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/09/25/colmlb-sfnmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-566547-2019-09-25',
 'season': '2019',
 'seasonDisplay': '2019'}

In [ ]:
GameDetails(**gameData['game'])

GameDetails(pk=566547, type=<GameType.regular_season: 'R'>, doubleHeader=False, id='2019/09/25/colmlb-sfnmlb-1', gamedayType=<GamedayType.premium: 'P'>, tiebreaker=False, gameNumber=1, calendarEventID='14-566547-2019-09-25', season='2019', seasonDisplay='2019')

## Game Datetime

In [ ]:
gameData['datetime']

{'dateTime': '2019-09-26T01:45:00Z',
 'originalDate': '2019-09-25',
 'dayNight': 'night',
 'time': '6:45',
 'ampm': 'PM'}

In [ ]:
#exporti 

class DayNight(str,Enum):
    day='day'
    night='night'
class AmPm(str,Enum):
    AM='AM'
    PM='PM'

In [ ]:
#exporti

class GameDatetime(BaseModel):
    dateTime: dt.datetime = Field(...,description="Timezone unaware datetime for the game")
    originalDate: dt.date
    dayNight: DayNight
    time: dt.time = Field(...,description='Local time for the game')
    ampm: AmPm
    
    @root_validator
    def validate_game_time(cls,values):
        """
        Adds 12 hours to the game time if 'pm'
        """
        if values['ampm']=='PM':
            values['time'] = dt.time(
                values['time'].hour + 12,
                values['time'].minute
            )
            return values
        return values

In [ ]:
gameData['datetime']

{'dateTime': '2019-09-26T01:45:00Z',
 'originalDate': '2019-09-25',
 'dayNight': 'night',
 'time': '6:45',
 'ampm': 'PM'}

In [ ]:
GameDatetime(**gameData['datetime'])

GameDatetime(dateTime=datetime.datetime(2019, 9, 26, 1, 45, tzinfo=datetime.timezone.utc), originalDate=datetime.date(2019, 9, 25), dayNight=<DayNight.night: 'night'>, time=datetime.time(18, 45), ampm=<AmPm.PM: 'PM'>)

## Game Status

In [ ]:
gameData['status']

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final',
 'statusCode': 'F',
 'startTimeTBD': False,
 'abstractGameCode': 'F'}

In [ ]:
game_statuses = mlb.get('meta',{'type':'gameStatus'})
game_statuses[0]

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final: Tie, decision by tiebreaker',
 'statusCode': 'FW',
 'reason': 'Tied (won in tiebreaker)',
 'abstractGameCode': 'F'}

In [ ]:
game_status_values = {
    k:set(x.get(k) for x in game_statuses)
    for k in gameData['status'].keys()
}

In [ ]:
game_status_values['abstractGameState']

{'Final', 'Live', 'Other', 'Preview'}

In [ ]:
game_status_values['abstractGameCode']

{'F', 'L', 'O', 'P'}

In [ ]:
#exporti

class GameStatus(BaseModel):
    abstractGameState: AbstractGameState
    codedGameState: constr(max_length=2)
    detailedState: str
    statusCode: constr(max_length=2)
    reason: Optional[str] = None 
    startTimeTBD: Optional[bool] = False
    abstractGameCode: AbstractGameCode

In [ ]:
gameData['status']

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final',
 'statusCode': 'F',
 'startTimeTBD': False,
 'abstractGameCode': 'F'}

In [ ]:
GameStatus(**gameData['status'])

GameStatus(abstractGameState=<AbstractGameState.Final: 'Final'>, codedGameState='F', detailedState='Final', statusCode='F', reason=None, startTimeTBD=False, abstractGameCode=<AbstractGameCode.F: 'F'>)

## Team

In [ ]:
gameData['teams'].keys()

dict_keys(['away', 'home'])

In [ ]:
team = gameData['teams']['home']
{k: list(v.keys()) for k,v in team.items() if type(v)==dict}

{'venue': ['id', 'name', 'link'],
 'springVenue': ['id', 'link'],
 'league': ['id', 'name', 'link'],
 'division': ['id', 'name', 'link'],
 'sport': ['id', 'link', 'name'],
 'record': ['gamesPlayed',
  'wildCardGamesBack',
  'leagueGamesBack',
  'springLeagueGamesBack',
  'sportGamesBack',
  'divisionGamesBack',
  'conferenceGamesBack',
  'leagueRecord',
  'records',
  'divisionLeader',
  'wins',
  'losses',
  'winningPercentage'],
 'springLeague': ['id', 'name', 'link', 'abbreviation']}

### Venue

Some of the values here are abridged versions of what the MLB API would return if called directly. For example, the `venue` enpoint returns name and id, but the springVenue value here just provides the id. 

In [ ]:
mlb.get('venue',{'venueIds':'4705'})

{'copyright': 'Copyright 2021 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'venues': [{'id': 4705,
   'name': 'Truist Park',
   'link': '/api/v1/venues/4705'}]}

In [ ]:
team['venue']

{'id': 2395, 'name': 'Oracle Park', 'link': '/api/v1/venues/2395'}

In [ ]:
team['springVenue']

{'id': 2532, 'link': '/api/v1/venues/2532'}

In [ ]:
mlb.get('venue',{'venueIds':'4705'})

{'copyright': 'Copyright 2021 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'venues': [{'id': 4705,
   'name': 'Truist Park',
   'link': '/api/v1/venues/4705'}]}

In [ ]:
#exporti 

class TeamVenueBase(MLBEndpointBase):
    id: int
class TeamVenue(TeamVenueBase):
    name: str

In [ ]:
TeamVenueBase(**team['springVenue'])

TeamVenueBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/2532', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/2532'), id=2532)

In [ ]:
TeamVenue(**team['venue'])

TeamVenue(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/2395', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/2395'), id=2395, name='Oracle Park')

### Record

In [ ]:
team['record']

{'gamesPlayed': 158,
 'wildCardGamesBack': '-',
 'leagueGamesBack': '-',
 'springLeagueGamesBack': '-',
 'sportGamesBack': '-',
 'divisionGamesBack': '-',
 'conferenceGamesBack': '-',
 'leagueRecord': {'wins': 76, 'losses': 82, 'pct': '.481'},
 'records': {},
 'divisionLeader': False,
 'wins': 76,
 'losses': 82,
 'winningPercentage': '.481'}

In [ ]:
from pydantic import confloat

In [ ]:
#exporti 

class TeamRecord(BaseModel):
    wins: conint(ge=0)
    losses: conint(ge=0)
    winningPercentage: confloat(ge=0,lt=1) = Field(...,alias='pct')
    

class GameTeamRecord(TeamRecord):
    gamesPlayed: int
    wildCardGamesBack: CustomInt
    leagueGamesBack: CustomInt
    springLeagueGamesBack: CustomInt
    sportGamesBack: CustomInt
    leagueRecord: TeamRecord
    divisionLeader: bool
    records: dict # not sure what's supposed to go here... 

    class Config:
        allow_population_by_field_name=True

In [ ]:
team['record']['leagueRecord']

{'wins': 76, 'losses': 82, 'pct': '.481'}

In [ ]:
TeamRecord(**team['record']['leagueRecord'])

TeamRecord(wins=76, losses=82, winningPercentage=0.481)

In [ ]:
team['record']

{'gamesPlayed': 158,
 'wildCardGamesBack': '-',
 'leagueGamesBack': '-',
 'springLeagueGamesBack': '-',
 'sportGamesBack': '-',
 'divisionGamesBack': '-',
 'conferenceGamesBack': '-',
 'leagueRecord': {'wins': 76, 'losses': 82, 'pct': '.481'},
 'records': {},
 'divisionLeader': False,
 'wins': 76,
 'losses': 82,
 'winningPercentage': '.481'}

In [ ]:
GameTeamRecord(**team['record'],by_alias=False)

GameTeamRecord(wins=76, losses=82, winningPercentage=0.481, gamesPlayed=158, wildCardGamesBack=None, leagueGamesBack=None, springLeagueGamesBack=None, sportGamesBack=None, leagueRecord=TeamRecord(wins=76, losses=82, winningPercentage=0.481), divisionLeader=False, records={})

In [ ]:
team.keys()

dict_keys(['id', 'name', 'link', 'season', 'venue', 'springVenue', 'teamCode', 'fileCode', 'abbreviation', 'teamName', 'locationName', 'firstYearOfPlay', 'league', 'division', 'sport', 'shortName', 'record', 'springLeague', 'allStarStatus', 'active'])

In [ ]:
team['sport']

{'id': 1, 'link': '/api/v1/sports/1', 'name': 'Major League Baseball'}

In [ ]:
mlb.get('sports',{'sportId':1})

{'copyright': 'Copyright 2021 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'sports': [{'id': 1,
   'code': 'mlb',
   'link': '/api/v1/sports/1',
   'name': 'Major League Baseball',
   'abbreviation': 'MLB',
   'sortOrder': 11,
   'activeStatus': True}]}

In [ ]:
team['division']

{'id': 203, 'name': 'National League West', 'link': '/api/v1/divisions/203'}

In [ ]:
import requests 

requests.get(mlb.BASE_URL + 'v1/divisions/204').json()

{'copyright': 'Copyright 2021 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'divisions': [{'id': 204,
   'name': 'National League East',
   'season': '2021',
   'nameShort': 'NL East',
   'link': '/api/v1/divisions/204',
   'abbreviation': 'NLE',
   'league': {'id': 104, 'link': '/api/v1/league/104'},
   'sport': {'id': 1, 'link': '/api/v1/sports/1'},
   'hasWildcard': False,
   'numPlayoffTeams': 1}]}

In [ ]:
#exporti 

class GameTeam(MLBEndpointReference):
    season: int
    venue: TeamVenue
    springVenue: TeamVenueBase
    teamCode: str
    fileCode: str
    abbreviation: str
    teamName: str
    locationName: str
    firstYearOfPlay: str
    league: MLBEndpointReference
    division: MLBEndpointReference
    shortName: str
    record: GameTeamRecord
    springLeague: MLBEndpointReference
    allStarStatus: bool
    active: bool
    
        

In [ ]:
game_team = GameTeam(**team)
game_team

GameTeam(link=HttpUrl('https://statsapi.mlb.com/api/v1/teams/137', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/teams/137'), id=137, name='San Francisco Giants', season=2019, venue=TeamVenue(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/2395', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/2395'), id=2395, name='Oracle Park'), springVenue=TeamVenueBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/2532', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/2532'), id=2532), teamCode='sfn', fileCode='sf', abbreviation='SF', teamName='Giants', locationName='San Francisco', firstYearOfPlay='1883', league=MLBEndpointReference(link=HttpUrl('https://statsapi.mlb.com/api/v1/league/104', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/league/104'), id=104, name='National League'), division=MLBEndpointReference(link=Ht

In [ ]:
game_team.springLeague

MLBEndpointReference(link=HttpUrl('https://statsapi.mlb.com/api/v1/league/114', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/league/114'), id=114, name='Cactus League')

In [ ]:
game_team.record

GameTeamRecord(wins=76, losses=82, winningPercentage=0.481, gamesPlayed=158, wildCardGamesBack=None, leagueGamesBack=None, springLeagueGamesBack=None, sportGamesBack=None, leagueRecord=TeamRecord(wins=76, losses=82, winningPercentage=0.481), divisionLeader=False, records={})

In [ ]:
game_teams = [GameTeam(**gameData['teams'][tm]) for tm in ('home','away')]

## Players

This is the full response from the `players` endpoint as well

In [ ]:
player_example = list(gameData['players'].values())[0]

In [ ]:
mlb.get('people',{'personIds':player_example['id']})['people'][0] == player_example

True

In [ ]:
player_example

{'id': 664041,
 'fullName': 'Jaylin Davis',
 'link': '/api/v1/people/664041',
 'firstName': 'Jaylin',
 'lastName': 'Davis',
 'primaryNumber': '49',
 'birthDate': '1994-07-01',
 'currentAge': 26,
 'birthCity': 'Greensboro',
 'birthStateProvince': 'NC',
 'birthCountry': 'USA',
 'height': '5\' 11"',
 'weight': 205,
 'active': True,
 'primaryPosition': {'code': '9',
  'name': 'Outfielder',
  'type': 'Outfielder',
  'abbreviation': 'RF'},
 'useName': 'Jaylin',
 'middleName': 'Malik',
 'boxscoreName': 'Davis, J',
 'gender': 'M',
 'isPlayer': True,
 'isVerified': True,
 'draftYear': 2015,
 'mlbDebutDate': '2019-09-04',
 'batSide': {'code': 'R', 'description': 'Right'},
 'pitchHand': {'code': 'R', 'description': 'Right'},
 'nameFirstLast': 'Jaylin Davis',
 'nameSlug': 'jaylin-davis-664041',
 'firstLastName': 'Jaylin Davis',
 'lastFirstName': 'Davis, Jaylin',
 'lastInitName': 'Davis, J',
 'initLastName': 'J Davis',
 'fullFMLName': 'Jaylin Malik Davis',
 'fullLFMName': 'Davis, Jaylin Malik',
 's

In [ ]:
positions = mlb.get('meta',{'type':'positions'})
positions[0]

{'shortName': 'Pitcher',
 'fullName': 'Pitcher',
 'abbrev': 'P',
 'code': '1',
 'type': 'Pitcher',
 'formalName': 'Pitcher',
 'pitcher': True,
 'gamePosition': True,
 'fielder': False,
 'outfield': False,
 'displayName': 'Pitcher'}

In [ ]:
player_example['primaryPosition']

{'code': '9', 'name': 'Outfielder', 'type': 'Outfielder', 'abbreviation': 'RF'}

In [ ]:
gameData['primaryDatacaster']

{'id': 496214, 'fullName': 'Karen Palmyra', 'link': '/api/v1/people/496214'}

In [ ]:
example_person = mlb.get('people',{'personIds':gameData['primaryDatacaster']['id']})['people'][0]
example_person

{'id': 496214,
 'fullName': 'Karen Palmyra',
 'link': '/api/v1/people/496214',
 'firstName': 'Karen',
 'lastName': 'Palmyra',
 'height': '0\' 0"',
 'weight': 0,
 'active': False,
 'primaryPosition': {'code': 'X',
  'name': 'Unknown',
  'type': 'Unknown',
  'abbreviation': 'X'},
 'useName': 'Karen',
 'boxscoreName': 'Palmyra',
 'gender': 'F',
 'isPlayer': False,
 'isVerified': False,
 'nameFirstLast': 'Karen Palmyra',
 'nameSlug': 'karen-palmyra-496214',
 'firstLastName': 'Karen Palmyra',
 'lastFirstName': 'Palmyra, Karen',
 'lastInitName': 'Palmyra, K',
 'initLastName': 'K Palmyra',
 'fullFMLName': 'Karen Palmyra',
 'fullLFMName': 'Palmyra, Karen'}

In [ ]:
person = MLBPerson(**example_person)

In [ ]:
person

MLBPerson(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/496214', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/496214'), id=496214, name='Karen Palmyra', firstName='Karen', lastName='Palmyra', height='0\' 0"', weight=0, active=False, primaryPosition=PositionBase(code='X', name='Unknown', type='Unknown', abbreviation='X'), useName='Karen', boxscoreName='Palmyra', gender='F', isPlayer=False, isVerified=False, nameSlug='karen-palmyra-496214')

In [ ]:
#exporti 

class Player(MLBPerson):
    primaryNumber: Optional[int] = None
    birthDate: dt.date
    currentAge: int
    birthCity: str
    birthStateProvince: Optional[str] = None
    birthCountry: str

    middleName: Optional[str] = None

    pitchHand: PlayerHandedness
    batSide: PlayerHandedness

    draftYear: Optional[int] = None
    mlbDebutDate: dt.date
    
    strikeZoneTop: float
    strikeZoneBottom: float
        

In [ ]:
player_example

{'id': 664041,
 'fullName': 'Jaylin Davis',
 'link': '/api/v1/people/664041',
 'firstName': 'Jaylin',
 'lastName': 'Davis',
 'primaryNumber': '49',
 'birthDate': '1994-07-01',
 'currentAge': 26,
 'birthCity': 'Greensboro',
 'birthStateProvince': 'NC',
 'birthCountry': 'USA',
 'height': '5\' 11"',
 'weight': 205,
 'active': True,
 'primaryPosition': {'code': '9',
  'name': 'Outfielder',
  'type': 'Outfielder',
  'abbreviation': 'RF'},
 'useName': 'Jaylin',
 'middleName': 'Malik',
 'boxscoreName': 'Davis, J',
 'gender': 'M',
 'isPlayer': True,
 'isVerified': True,
 'draftYear': 2015,
 'mlbDebutDate': '2019-09-04',
 'batSide': {'code': 'R', 'description': 'Right'},
 'pitchHand': {'code': 'R', 'description': 'Right'},
 'nameFirstLast': 'Jaylin Davis',
 'nameSlug': 'jaylin-davis-664041',
 'firstLastName': 'Jaylin Davis',
 'lastFirstName': 'Davis, Jaylin',
 'lastInitName': 'Davis, J',
 'initLastName': 'J Davis',
 'fullFMLName': 'Jaylin Malik Davis',
 'fullLFMName': 'Davis, Jaylin Malik',
 's

In [ ]:
Player(**player_example)

Player(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/664041', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/664041'), id=664041, name='Jaylin Davis', firstName='Jaylin', lastName='Davis', height='5\' 11"', weight=205, active=True, primaryPosition=PositionBase(code='9', name='Outfielder', type='Outfielder', abbreviation='RF'), useName='Jaylin', boxscoreName='Davis, J', gender='M', isPlayer=True, isVerified=True, nameSlug='jaylin-davis-664041', primaryNumber=49, birthDate=datetime.date(1994, 7, 1), currentAge=26, birthCity='Greensboro', birthStateProvince='NC', birthCountry='USA', middleName='Malik', pitchHand=PlayerHandedness(code='R', description='Right'), batSide=PlayerHandedness(code='R', description='Right'), draftYear=2015, mlbDebutDate=datetime.date(2019, 9, 4), strikeZoneTop=3.319, strikeZoneBottom=1.513)

In [ ]:
players = {k:Player(**v) for k,v in gameData['players'].items()}

## Venue

In [ ]:
venues = pd.read_sql_table('venue',engine.connect())

print(venues['fieldInfo_turfType'].unique())
print(venues['fieldInfo_roofType'].unique())

['Grass' 'Artificial']
['Open' 'Dome' 'Retractable']


In [ ]:
gameData['venue']

{'id': 2395,
 'name': 'Oracle Park',
 'link': '/api/v1/venues/2395',
 'location': {'city': 'San Francisco',
  'state': 'California',
  'stateAbbrev': 'CA',
  'defaultCoordinates': {'latitude': 37.778383, 'longitude': -122.389448}},
 'timeZone': {'id': 'America/Los_Angeles', 'offset': -8, 'tz': 'PST'},
 'fieldInfo': {'capacity': 41915,
  'turfType': 'Grass',
  'roofType': 'Open',
  'leftLine': 339,
  'leftCenter': 364,
  'center': 399,
  'rightCenter': 421,
  'rightLine': 309}}

In [ ]:
#exporti 

class GameVenue(TeamVenue):
    location: VenueLocation
    timeZone: TimeZone
    fieldInfo: FieldInfo

In [ ]:
GameVenue(**gameData['venue'])

GameVenue(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/2395', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/2395'), id=2395, name='Oracle Park', location=VenueLocation(city='San Francisco', state='California', stateAbbrev='CA', defaultCoordinates=Coordinates(latitude=37.778383, longitude=-122.389448)), timeZone=TimeZone(id='America/Los_Angeles', offset=-8, tz='PST'), fieldInfo=FieldInfo(capacity=41915, turfType=<TurfType.Grass: 'Grass'>, roofType=<RoofType.Open: 'Open'>, leftLine=339, leftCenter=364, center=399, rightCenter=421, rightLine=309))

## Weather

In [ ]:
gameData['weather']

{'condition': 'Clear', 'temp': '83', 'wind': '11 mph, Out To CF'}

In [ ]:
mlb.get('meta',{'type':'windDirection'})

[{'code': 'Calm', 'description': 'Calm'},
 {'code': 'None', 'description': 'None'},
 {'code': 'Varies', 'description': 'Varies'},
 {'code': 'In From RF', 'description': 'In From RF'},
 {'code': 'In From LF', 'description': 'In From LF'},
 {'code': 'In From CF', 'description': 'In From CF'},
 {'code': 'R To L', 'description': 'R To L'},
 {'code': 'L To R', 'description': 'L To R'},
 {'code': 'Out To CF', 'description': 'Out To CF'},
 {'code': 'Out To RF', 'description': 'Out To RF'},
 {'code': 'Out To LF', 'description': 'Out To LF'}]

In [ ]:
#exporti 

class GameWeather(BaseModel):
    condition: str
    temp: int
    wind: str

In [ ]:
GameWeather(**gameData['weather'])

GameWeather(condition='Clear', temp=83, wind='11 mph, Out To CF')

In [ ]:
gameData['gameInfo']

{'attendance': 29175, 'gameDurationMinutes': 193}

## Game Info

In [ ]:
#exporti 

class GameInfo(BaseModel):
    attendance: int
    gameDurationMinutes: int
    
    @property
    def game_duration(self):
        return dt.timedelta(minutes=self.gameDurationMinutes)

In [ ]:
game_info = GameInfo(**gameData['gameInfo'])
game_info.game_duration

datetime.timedelta(seconds=11580)

In [ ]:
duration = game_info.game_duration

## Review

In [ ]:
gameData['review']

{'hasChallenges': True,
 'away': {'used': 0, 'remaining': 1},
 'home': {'used': 1, 'remaining': 0}}

In [ ]:
#exporti 

class Challenges(BaseModel):
    used: int
    remaining: int

class GameReview(BaseModel):
    hasChallenges: bool
    away: Challenges
    home: Challenges

In [ ]:
GameReview(**gameData['review'])

GameReview(hasChallenges=True, away=Challenges(used=0, remaining=1), home=Challenges(used=1, remaining=0))

## Flags

In [ ]:
gameData['flags']

{'noHitter': False,
 'perfectGame': False,
 'awayTeamNoHitter': False,
 'awayTeamPerfectGame': False,
 'homeTeamNoHitter': False,
 'homeTeamPerfectGame': False}

In [ ]:
#exporti 

class GameFlags(BaseModel):
    noHitter: bool
    perfectGame: bool
    awayTeamNoHitter: bool
    awayTeamPerfectGame: bool
    homeTeamNoHitter: bool
    homeTeamPerfectGame: bool

In [ ]:
GameFlags(**gameData['flags'])

GameFlags(noHitter=False, perfectGame=False, awayTeamNoHitter=False, awayTeamPerfectGame=False, homeTeamNoHitter=False, homeTeamPerfectGame=False)

In [ ]:
gameData.keys()

dict_keys(['game', 'datetime', 'status', 'teams', 'players', 'venue', 'weather', 'gameInfo', 'review', 'flags', 'alerts', 'probablePitchers', 'officialScorer', 'primaryDatacaster'])

## Probable Pitchers, official Scorer, datacaster

In [ ]:
gameData['probablePitchers']

{'away': {'id': 543532,
  'fullName': 'Tim Melville',
  'link': '/api/v1/people/543532'},
 'home': {'id': 502188,
  'fullName': 'Jeff Samardzija',
  'link': '/api/v1/people/502188'}}

In [ ]:
[PersonBase(**gameData['probablePitchers'][tm]) for tm in ('home','away')]

[PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/502188', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/502188'), id=502188, name='Jeff Samardzija'),
 PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/543532', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/543532'), id=543532, name='Tim Melville')]

In [ ]:
PersonBase(**gameData['officialScorer'])

PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/639285', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/639285'), id=639285, name='Chris Thoms')

In [ ]:
PersonBase(**gameData['primaryDatacaster'])

PersonBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/people/496214', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/people/496214'), id=496214, name='Karen Palmyra')

## GameData

In [ ]:
gameData.keys()

dict_keys(['game', 'datetime', 'status', 'teams', 'players', 'venue', 'weather', 'gameInfo', 'review', 'flags', 'alerts', 'probablePitchers', 'officialScorer', 'primaryDatacaster'])

In [ ]:
#exporti 

class HomeAway(str,Enum):
    home = 'home'
    away = 'away'

In [ ]:
#export 

class GameData(BaseModel):
    game: GameDetails
    datetime: GameDatetime
    status: GameStatus
    teams: Dict[HomeAway,GameTeam]
    players: Dict[str,Player]
    venue: GameVenue
    gameInfo: GameInfo
    review: GameReview
    flags: GameFlags
    probablePitchers: Dict[HomeAway,PersonBase]
    officialScorer: PersonBase
    primaryDatacaster: PersonBase

In [ ]:
game_data = GameData(**gameData)

In [ ]:
game_data.teams['away']

GameTeam(link=HttpUrl('https://statsapi.mlb.com/api/v1/teams/115', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/teams/115'), id=115, name='Colorado Rockies', season=2019, venue=TeamVenue(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/19', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/19'), id=19, name='Coors Field'), springVenue=TeamVenueBase(link=HttpUrl('https://statsapi.mlb.com/api/v1/venues/4249', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/venues/4249'), id=4249), teamCode='col', fileCode='col', abbreviation='COL', teamName='Rockies', locationName='Denver', firstYearOfPlay='1992', league=MLBEndpointReference(link=HttpUrl('https://statsapi.mlb.com/api/v1/league/104', scheme='https', host='statsapi.mlb.com', tld='com', host_type='domain', path='/api/v1/league/104'), id=104, name='National League'), division=MLBEndpointReference(link=HttpUrl('https:/

In [ ]:
GameData.schema()

{'title': 'GameData',
 'type': 'object',
 'properties': {'game': {'$ref': '#/definitions/GameDetails'},
  'datetime': {'$ref': '#/definitions/GameDatetime'},
  'status': {'$ref': '#/definitions/GameStatus'},
  'teams': {'title': 'Teams',
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/GameTeam'}},
  'players': {'title': 'Players',
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/Player'}},
  'venue': {'$ref': '#/definitions/GameVenue'},
  'gameInfo': {'$ref': '#/definitions/GameInfo'},
  'review': {'$ref': '#/definitions/GameReview'},
  'flags': {'$ref': '#/definitions/GameFlags'},
  'probablePitchers': {'title': 'Probablepitchers',
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/PersonBase'}},
  'officialScorer': {'$ref': '#/definitions/PersonBase'},
  'primaryDatacaster': {'$ref': '#/definitions/PersonBase'}},
 'required': ['game',
  'datetime',
  'status',
  'teams',
  'players',
  'venue',
  'gameInfo',
  'revie

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_mlbAPI.ipynb.
Converted 02_application.ipynb.
Converted 03_models.base_models.ipynb.
Converted 03a_models.game_data.ipynb.
Converted 04_database.ipynb.
Converted index.ipynb.
